In [3]:
import numpy as np
import pandas as pd

In [10]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from sklearn.metrics.pairwise import cosine_similarity

ERROR! Session/line number was not unique in database. History logging moved to new session 16


In [5]:
df = pd.read_csv('1.csv')
df = df[['name','rate','location','dish_liked','cuisines','approx_cost','neighbourhood']]
df.drop_duplicates('name', inplace=True)
df.dropna(subset=['cuisines'], inplace=True)

In [6]:
import string
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

df["cuisines"] = df["cuisines"].apply(lambda text: remove_punctuation(text))
df['addr'] = df['neighbourhood'] +' '+ df['location']
df.set_index('name', inplace=True)
indices = pd.Series(df.index)

In [69]:
df

,rate,location,dish_liked,cuisines,approx_cost,neighbourhood,addr
name,,,,,,,
Jalsa,4.1,Banashankari,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...",North Indian Mughlai Chinese,800.0,Banashankari,Banashankari Banashankari
Spice Elephant,4.1,Banashankari,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...",Chinese North Indian Thai,800.0,Banashankari,Banashankari Banashankari
San Churro Cafe,3.8,Banashankari,"Churros, Cannelloni, Minestrone Soup, Hot Choc...",Cafe Mexican Italian,800.0,Banashankari,Banashankari Banashankari
Addhuri Udupi Bhojana,3.7,Banashankari,Masala Dosa,South Indian North Indian,300.0,Banashankari,Banashankari Banashankari
Grand Village,3.8,Basavanagudi,"Panipuri, Gol Gappe",North Indian Rajasthani,600.0,Banashankari,Banashankari Basavanagudi
...,...,...,...,...,...,...,...
Chime - Sheraton Grand Bengaluru Whitefield Hotel &...,4.3,"ITPL Main Road, Whitefield","Cocktails, Pizza, Buttermilk",Finger Food,2500.0,Whitefield,"Whitefield ITPL Main Road, Whitefield"
The Nest - The Den Bengaluru,3.4,"ITPL Main Road, Whitefield",NaN,Finger Food North Indian Continental,1500.0,Whitefield,"Whitefield ITPL Main Road, Whitefield"
Nawabs Empire,3.2,"ITPL Main Road, Whitefield",NaN,North Indian Chinese Arabian Momos,300.0,Whitefield,"Whitefield ITPL Main Road, Whitefield"


In [70]:
l = [i for i in df['addr']]
l2 = [i for i in df['cuisines']]
l3 = [i for i in df['dish_liked']]

In [122]:
dish=[]
for i in l3:
    if pd.isna(i):
        dish.append('')
    else:
        dish.append(''.join(i.split(',')))

In [90]:
# integer encode the documents
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in l]
encoded_docs2 = [one_hot(d, vocab_size) for d in l2]
encoded_docs3 = [one_hot(d, vocab_size) for d in dish]

max_length = 10
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
padded_docs2 = pad_sequences(encoded_docs2, maxlen=max_length, padding='post')
padded_docs3 = pad_sequences(encoded_docs3, maxlen=max_length, padding='post')


In [154]:
# define the model
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(Flatten())
model.add(Dense(4, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 10, 8)             400       
_________________________________________________________________
flatten_3 (Flatten)          (None, 80)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 324       
Total params: 724
Trainable params: 724
Non-trainable params: 0
_________________________________________________________________
None


In [155]:
# fit the model
embd = model.predict(padded_docs)
embd2 = model.predict(padded_docs2)
embd3 = model.predict(padded_docs3)

In [167]:
embd,embd2,embd3

(array([[0.51117355, 0.5022387 , 0.50534844, 0.48515952],
        [0.51117355, 0.5022387 , 0.50534844, 0.48515952],
        [0.51117355, 0.5022387 , 0.50534844, 0.48515952],
        ...,
        [0.50203747, 0.4995967 , 0.49581507, 0.4913889 ],
        [0.50924534, 0.50173354, 0.50335324, 0.4904017 ],
        [0.5125977 , 0.5012968 , 0.5060778 , 0.48626697]], dtype=float32),
 array([[0.5178398 , 0.5032603 , 0.51849806, 0.48774347],
        [0.51091754, 0.49033064, 0.50166166, 0.4971073 ],
        [0.5106723 , 0.5026635 , 0.50419295, 0.4908711 ],
        ...,
        [0.50861484, 0.50023675, 0.50998384, 0.4847136 ],
        [0.5200511 , 0.49330115, 0.5094587 , 0.48572958],
        [0.5165525 , 0.500414  , 0.49841112, 0.4753526 ]], dtype=float32),
 array([[0.49649718, 0.48879308, 0.4919437 , 0.49078986],
        [0.5046487 , 0.49481007, 0.50824535, 0.49910063],
        [0.49673957, 0.47620332, 0.5022186 , 0.49727875],
        ...,
        [0.5185101 , 0.50078243, 0.504731  , 0.48714682],

In [185]:
sim,sim2,sim3 = [],[],[]
for i in range(indices.shape[0]):
    #sim.append([embd[i][0],embd2[i][0],embd3[i][0]])
    sim.append(list(embd[i]))
    sim2.append(list(embd2[i]))
    sim3.append(list(embd3[i]))  

In [149]:
[sim[0]]

[[0.4851062]]

In [186]:
def rec(name):
    idx = indices[indices == name].index[0]
    corr,corr2,corr3 = [],[],[]
    for i in range(len(sim)):
        corr.append( (cosine_similarity([sim[idx]],[sim[i]])[0][0], i))
        corr2.append( (cosine_similarity([sim2[idx]],[sim2[i]])[0][0], i))
        corr3.append( (cosine_similarity([sim3[idx]],[sim3[i]])[0][0], i))
    corr = sorted(corr,key = lambda x:x[0], reverse=True)[:1000]
    corr2 = sorted(corr2,key = lambda x:x[0], reverse=True)[:1000]
    corr3 = sorted(corr3,key = lambda x:x[0], reverse=True)[:1000]
    corr = [c[1] for c in corr]
    corr2 = [c[1] for c in corr2]
    corr3 = [c[1] for c in corr3]
    
    reco=list(set(corr).intersection(corr2).intersection(corr3))[:10]
    
    return df.iloc[reco]
    

    
rec('Jalsa')

,rate,location,dish_liked,cuisines,approx_cost,neighbourhood,addr
name,,,,,,,
Jalsa,4.1,Banashankari,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...",North Indian Mughlai Chinese,800.0,Banashankari,Banashankari Banashankari
Skylit - Citrus Hotels,3.6,Bellandur,"Chicken Curry, Mocktails, Biryani, Halwa, Beer...",North Indian Mughlai Mediterranean,1500.0,Bellandur,Bellandur Bellandur
Hara Fine Dine,4.0,Banashankari,"Hara Bhara Kabab, Crispy Corn, Patiala Paneer,...",North Indian Chinese,800.0,Banashankari,Banashankari Banashankari
Farzi Cafe,4.4,Lavelle Road,"Appletini, Lamb Tacos, Cocktails, Martini, Sto...",Modern Indian,1500.0,Brigade Road,Brigade Road Lavelle Road
Rangla Punjab,2.9,Bellandur,"Naan, Paratha, Sweet Lassi, Egg Masala, Matar ...",North Indian Chinese,450.0,Bellandur,Bellandur Bellandur
Caperberry,4.6,Lavelle Road,"Salads, Martini, Tiramisu, Caviar, Ravioli, Co...",Spanish Italian Mediterranean European Salad,2200.0,Brigade Road,Brigade Road Lavelle Road
Rasoi,3.3,Banashankari,"Lassi, Panneer Butter Masala, Paratha, Thali",North Indian Chinese,300.0,Banashankari,Banashankari Banashankari
Punjabi Dawat,2.9,Bellandur,"Lassi, Naan, Chicken Biryani, Butter Chicken, ...",North Indian Chinese,400.0,Bellandur,Bellandur Bellandur
Sigree Global Grill,4.4,Kalyan Nagar,"Pasta, Mutton Seekh, Fish, Prawn Curry, Pizza,...",North Indian BBQ,1400.0,Kalyan Nagar,Kalyan Nagar Kalyan Nagar


In [170]:
cosine_similarity([[11,1]],[[52,2]])[0][0]

0.9986369866844395